In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense , MaxPooling2D , Dropout , Conv2D , BatchNormalization , Flatten
from keras.models import Sequential
from keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [2]:
from PIL import Image
import os

images_palm = []
images_i = []
images_fist = []
images_fist_moved = []
images_palm_moved = []
images_c = []
images_down = []
images_ok = []
images_index = []
images_thumb = []

def read_all_images(directory , images):
    if os.path.exists(directory):
        for filename in os.listdir(directory):
            if filename.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif')):
                image_path = os.path.join(directory, filename)
                img = image.load_img(image_path , target_size=(128, 128, 1), color_mode='grayscale')
                img = image.img_to_array(img)
                img = img / 255
                images.append(img)
    else:
        print(f"The directory {directory} does not exist.")
    return images

In [3]:
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/01_palm'
    read_all_images(directory , images_palm)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/02_l'
    read_all_images(directory , images_i)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/03_fist'
    read_all_images(directory , images_fist)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/04_fist_moved'
    read_all_images(directory , images_fist_moved)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/05_thumb'
    read_all_images(directory , images_thumb)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/06_index'
    read_all_images(directory , images_index)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/07_ok'
    read_all_images(directory , images_ok)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/08_palm_moved'
    read_all_images(directory , images_palm_moved)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/09_c'
    read_all_images(directory , images_c)
for i in range(0,7):
    directory = './inages/' + '0' + str(i) + '/10_down'
    read_all_images(directory , images_down)

In [4]:
value = 0
label_palm = [value]*len(images_palm)
value = 1 
label_i = [value]*len(images_i)
value = 2
label_first = [value]*len(images_fist)
value = 3
label_first_moved = [value]*len(images_fist_moved)
value = 4
label_thumb = [value]*len(images_thumb)
value = 5 
label_index = [value]*len(images_index)
value = 6 
label_ok = [value]*len(images_ok)
value = 7 
label_palm_moved = [value]*len(images_palm_moved)
value = 8 
label_c = [value]*len(images_c)
value = 9 
label_down = [value]*len(images_down)

In [5]:
image_list =images_palm + images_i + images_fist + images_fist_moved + images_palm_moved + images_c +images_down + images_ok +images_index +images_thumb 

In [6]:
label_list =  label_palm + label_i + label_first + label_first_moved + label_palm_moved + label_c + label_down + label_ok + label_index + label_thumb

In [7]:
image_array = np.array(image_list)
label_array = np.array(label_list)

In [8]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_train , x_val , train_out , val_out = train_test_split(image_array , label_array , test_size = 0.2 , random_state = 42)

In [22]:
x_train = x_train.astype(np.float16)
x_val = x_val.astype(np.float16)

In [24]:
from keras.utils import to_categorical

In [26]:
train_out = to_categorical(train_out)
val_out = to_categorical(val_out)

In [28]:
with strategy.scope():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3,3) , strides = 1,padding = 'same', input_shape = (128,128,1),activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(4,4) , strides = 1,padding = 'same' , activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(1024 , activation = 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(10, activation = 'softmax'))

    model.compile(optimizer = tf.keras.optimizers.RMSprop(0.0001), # 1e-4
              loss = 'categorical_crossentropy', # Ideal for multiclass tasks
              metrics = ['accuracy']) # Evaluation metric

    early_stopping = EarlyStopping(monitor = 'val_accuracy',
                              patience = 5, mode = 'max',
                              restore_best_weights = True)

    checkpoint = ModelCheckpoint('best_model.keras',
                            monitor = 'val_accuracy',
                            save_best_only = True)
    model.fit(x_train , train_out , epochs=10 , validation_data=(x_val , val_out))
    

Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 228s 644ms/step - accuracy: 0.6808 - loss: 3.1270 - val_accuracy: 0.0921 - val_loss: 11.4926
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 201s 575ms/step - accuracy: 0.9791 - loss: 0.0835 - val_accuracy: 0.9654 - val_loss: 0.1039
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 193s 550ms/step - accuracy: 0.9937 - loss: 0.0236 - val_accuracy: 1.0000 - val_loss: 2.4295e-04
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 194s 552ms/step - accuracy: 0.9972 - loss: 0.0095 - val_accuracy: 1.0000 - val_loss: 2.3812e-04
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 197s 562ms/step - accuracy: 0.9987 - loss: 0.0038 - val_accuracy: 0.9996 - val_loss: 6.5449e-04
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 198s 565ms/step - accuracy: 0.9995 - loss: 0.0017 - val_accuracy: 0.9993 - val_loss: 0.0015
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 210s 600ms/step - accuracy: 0.9994 - loss: 0.0011 - val_accuracy: 0.9989 - val_loss: 0.0036
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 210s 599ms/step - accur

In [30]:
model.save("cnn_model.keras")

In [32]:
np.save('image_array.npy' , image_array)

In [33]:
np.save('label_array.npy' , label_array)

testing


In [39]:
test_images_palm = []
test_images_i = []
test_images_fist = []
test_images_fist_moved = []
test_images_thumb = []
test_images_index = []
test_images_down = []
test_images_c = []
test_images_palm_moved = []
test_images_ok = []

for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/01_palm'
    read_all_images(directory , test_images_palm)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/02_l'
    read_all_images(directory , test_images_i)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/03_fist'
    read_all_images(directory , test_images_fist)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/04_fist_moved'
    read_all_images(directory , test_images_fist_moved)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/05_thumb'
    read_all_images(directory , test_images_thumb)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/06_index'
    read_all_images(directory , test_images_index)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/07_ok'
    read_all_images(directory , test_images_ok)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/08_palm_moved'
    read_all_images(directory , test_images_palm_moved)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/09_c'
    read_all_images(directory , test_images_c)
for i in range(8,9):
    directory = './inages/' + '0' + str(i) + '/10_down'
    read_all_images(directory , test_images_down)
    

In [44]:
value = 0
test_label_palm = [value]*len(test_images_palm)
value = 1 
test_label_i = [value]*len(test_images_i)
value = 2
test_label_first = [value]*len(test_images_fist)
value = 3
test_label_first_moved = [value]*len(test_images_fist_moved)
value = 4
test_label_thumb = [value]*len(test_images_thumb)
value = 5 
test_label_index = [value]*len(test_images_index)
value = 6 
test_label_ok = [value]*len(test_images_ok)
value = 7 
test_label_palm_moved = [value]*len(test_images_palm_moved)
value = 8 
test_label_c = [value]*len(test_images_c)
value = 9 
test_label_down = [value]*len(test_images_down)

In [45]:
test_label_list =  test_label_palm + test_label_i + test_label_first + test_label_first_moved + test_label_palm_moved + test_label_c + test_label_down + test_label_ok + test_label_index + test_label_thumb

In [40]:
test_image_list =test_images_palm + test_images_i + test_images_fist + test_images_fist_moved + test_images_palm_moved + test_images_c +test_images_down + test_images_ok +test_images_index +test_images_thumb 

In [42]:
test_image_array = np.array(test_image_list)

In [47]:
predict = model.predict(test_image_array)
pred_classes = np.argmax(predict, axis=1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step


In [48]:
from sklearn.metrics import classification_report, confusion_matrix

In [49]:
print("Classification Report:")
print(classification_report(test_label_list, pred_classes))

print("Confusion Matrix:")
print(confusion_matrix(test_label_list, pred_classes))

Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.50      0.40       200
           1       0.70      0.67      0.68       200
           2       0.69      0.30      0.42       200
           3       0.00      0.00      0.00       200
           4       0.42      0.12      0.19       200
           5       0.44      0.72      0.55       200
           6       0.11      0.02      0.03       200
           7       0.47      1.00      0.64       200
           8       0.38      0.35      0.36       200
           9       0.56      1.00      0.72       200

    accuracy                           0.47      2000
   macro avg       0.41      0.47      0.40      2000
weighted avg       0.41      0.47      0.40      2000

Confusion Matrix:
[[100   0  11  13  14  12   7  11   2  30]
 [  6 133  17  10  10   2   0   6   3  13]
 [ 20   0  61   7   9  36  24   0  19  24]
 [ 84   0   0   0   0   8   0  16  92   0]
 [ 80   0   0   0  24  96   